<a href="https://colab.research.google.com/github/Festuskipkoech/Festus_data-science/blob/main/RecommendationSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Recommendation Systems
This are applications used in various platforms to suggest the likelihood of a customer preference e,g Ecommerce we can suggest a likelihood of a customer purchasing a certain product, In news articles we can also recommend a likehood of person reading a particular news etc. We can two major categories of recommendation:

A recommendation system typically provides personalized suggestions to users based on certain algorithms. Here’s an overview of the two types you mentioned:

1. Collaborative Filtering
Collaborative filtering is based on user interaction data and relies on the preferences of similar users or items. It's split into two main approaches:

User-based Collaborative Filtering: Recommends items to a user based on what similar users have liked. For example, if User A and User B have similar interests, items that User B has liked but User A hasn't seen yet will be recommended to User A.

Item-based Collaborative Filtering: Recommends items to a user based on the similarity between items. If a user likes an item, the system will recommend similar items. For example, if User A likes a specific book, they might be recommended books with similar genres or authors that other users have liked after reading the same book.

This approach requires a large amount of user interaction data (ratings, clicks, views) and works well when there is a high user-to-user interaction.

Advantages:

Effective when user-item interaction data is available.
Automatically adapts to changes in user preferences.
Challenges:

Cold start problem: Difficult to recommend for new users or items with little to no data.
Scalability can become an issue with a large dataset.
2. Content-Based Filtering
Content-based filtering recommends items to a user by comparing the content or attributes of items with the user's past interactions or preferences. Each item and user is represented by feature vectors, and the system recommends items with attributes that are most similar to those the user has interacted with in the past.

For example, in a movie recommendation system, if a user has previously watched and enjoyed action movies, the system will recommend other movies that have similar genres, directors, actors, etc.

Steps:

Build a user profile by tracking the features of items they’ve interacted with.
Compare the profile with the features of other items in the system to find matches.
Advantages:

Can recommend items even if no user interaction data is available (helps solve the cold start problem).
Recommendations are based on clear, interpretable features (e.g., genre, director).
Challenges:

Limited to the features provided for the items.
Over-specialization: Recommends items similar to what the user has already seen, which might limit diversity.


In [ ]:
# Movie recommendation system
# IMDB dataset
# 1. Reading the users movie dataset



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
# users dataset
users = pd.read_csv('https://msi.martial.co.ke/data/file.csv')
users

,user_id,item_id,rating,timestamp
0,0,50,5,881250949
1,0,172,5,881250949
2,0,133,1,881250949
3,196,242,3,881250949
4,186,302,3,891717742
...,...,...,...,...
99998,880,476,3,880175444
99999,716,204,5,879795543
100000,276,1090,1,874795795
100001,13,225,2,882399156


In [3]:

# Movie IDS Dataset
movies = pd.read_csv('https://msi.martial.co.ke/data/Movie_Id_Titles.csv')
movies

,item_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)
...,...,...
1677,1678,Mat' i syn (1997)
1678,1679,B. Monkey (1998)
1679,1680,Sliding Doors (1998)
1680,1681,You So Crazy (1994)


In [4]:
# Merge the users and movies Dataset on item_id
df = pd.merge(users,movies, on='item_id')
df

,user_id,item_id,rating,timestamp,title
0,0,50,5,881250949,Star Wars (1977)
1,0,172,5,881250949,"Empire Strikes Back, The (1980)"
2,0,133,1,881250949,Gone with the Wind (1939)
3,196,242,3,881250949,Kolya (1996)
4,186,302,3,891717742,L.A. Confidential (1997)
...,...,...,...,...,...
99998,880,476,3,880175444,"First Wives Club, The (1996)"
99999,716,204,5,879795543,Back to the Future (1985)
100000,276,1090,1,874795795,Sliver (1993)
100001,13,225,2,882399156,101 Dalmatians (1996)


In [5]:
# Exploratory Data Analysis
# 1. Information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100003 entries, 0 to 100002
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   user_id    100003 non-null  int64 
 1   item_id    100003 non-null  int64 
 2   rating     100003 non-null  int64 
 3   timestamp  100003 non-null  int64 
 4   title      100003 non-null  object
dtypes: int64(4), object(1)
memory usage: 3.8+ MB


In [6]:
# 2. Maximium and Minimum Rating and Avearage Rating
print(df['rating'].max())
print(df['rating'].min())
print(df['rating'].mean())
print(df['title'].count())

5
1
3.5298641040768777
100003


In [8]:
# 3. Statistical Analysis
# Total Users: 943
# Total Movies: 1682
df.describe()

,user_id,item_id,rating,timestamp
count,100003.000000,100003.000000,100003.000000,1.000030e+05
mean,462.470876,425.520914,3.529864,8.835288e+08
std,266.622454,330.797791,1.125704,5.343791e+06
min,0.000000,1.000000,1.000000,8.747247e+08
25%,254.000000,175.000000,3.000000,8.794487e+08
50%,447.000000,322.000000,4.000000,8.828269e+08
75%,682.000000,631.000000,4.000000,8.882600e+08
max,943.000000,1682.000000,5.000000,8.932866e+08


In [9]:
# 4.Checking the Empties
df.isnull().sum()

,0
user_id,0
item_id,0
rating,0
timestamp,0
title,0


In [12]:
# RECOMMENDATION SYSTEM : COLLABORATIVE FILTERING
# Grouping All the Movies Tiltles According to The Average Rating
mean_ratings = df.groupby('title')['rating'].mean()
mean_ratings

# Top 10 Rated Movies
mean_ratings.sort_values(ascending=False).head(10)

,rating
title,
They Made Me a Criminal (1939),5.0
Marlene Dietrich: Shadow and Light (1996),5.0
"Saint of Fort Washington, The (1993)",5.0
Someone Else's America (1995),5.0
Star Kid (1997),5.0
"Great Day in Harlem, A (1994)",5.0
Aiqing wansui (1994),5.0
Santa with Muscles (1996),5.0
Prefontaine (1997),5.0


In [14]:

# How Many Users Rated a Particular Movie Item(Popularity)
rating_count = df.groupby('title')['rating'].count()
rating_count

,rating
title,
'Til There Was You (1997),9
1-900 (1994),5
101 Dalmatians (1996),109
12 Angry Men (1957),125
187 (1997),41
...,...
Young Guns II (1990),44
"Young Poisoner's Handbook, The (1995)",41
Zeus and Roxanne (1997),6


In [15]:
# Top Ten Movies Based on Popularity
rating_count.sort_values(ascending=False).head(10)

,rating
title,
Star Wars (1977),584
Contact (1997),509
Fargo (1996),508
Return of the Jedi (1983),507
Liar Liar (1997),485
"English Patient, The (1996)",481
Scream (1996),478
Toy Story (1995),452
Air Force One (1997),431


In [18]:
# Create a DataFrame that will Contain, the title, mean_rating, number_of_views
df_new = pd.DataFrame(mean_ratings)
df_new
df_new['number_of_views'] = pd.DataFrame(rating_count)
df_new.sort_values(by='number_of_views', ascending=False)


,rating,number_of_views
title,,
Star Wars (1977),4.359589,584
Contact (1997),3.803536,509
Fargo (1996),4.155512,508
Return of the Jedi (1983),4.007890,507
Liar Liar (1997),3.156701,485
...,...,...
"Great Day in Harlem, A (1994)",5.000000,1
"Other Voices, Other Rooms (1997)",3.000000,1
Good Morning (1971),1.000000,1


In [19]:
# PIVOT TABLE
# Data can be Stored in 3D
# X-Columns
# Y-indexing
# Z-Values
pivot = df.pivot_table(index='user_id', columns='title', values='rating')
pivot

title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
user_id,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,2.0,5.0,NaN,NaN,3.0,4.0,NaN,NaN,...,NaN,NaN,NaN,5.0,3.0,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
selected_movie = pivot['Godfather, The (1972)']
selected_movie

,"Godfather, The (1972)"
user_id,
0,NaN
1,5.0
2,5.0
3,NaN
4,NaN
...,...
939,5.0
940,NaN
941,NaN


In [23]:
import warnings
warnings.filterwarnings('ignore')
similar = pivot.corrwith(selected_movie)
similar_df = pd.DataFrame(similar, columns = ['Correlation'])
similar_df.sort_values('Correlation', ascending=False)

,Correlation
title,
Dark City (1998),1.0
8 Seconds (1994),1.0
Talking About Sex (1994),1.0
"Godfather, The (1972)",1.0
"Old Lady Who Walked in the Sea, The (Vieille qui marchait dans la mer, La) (1991)",1.0
...,...
Wonderland (1997),NaN
"Wooden Man's Bride, The (Wu Kui) (1994)",NaN
Yankee Zulu (1994),NaN


In [24]:
# Combine the Correlations and Popularity to make a Solid Collaborative Filtering
similar_df = similar_df.join(df_new['number_of_views'])
similar_df

,Correlation,number_of_views
title,,
'Til There Was You (1997),0.612372,9
1-900 (1994),-0.471405,5
101 Dalmatians (1996),0.084183,109
12 Angry Men (1957),0.034258,125
187 (1997),0.467335,41
...,...,...
Young Guns II (1990),0.129426,44
"Young Poisoner's Handbook, The (1995)",0.059479,41
Zeus and Roxanne (1997),-0.944911,6


In [28]:
# Recommendation:
# Good Positive Correlation
# Higher Number of Views(More than 200 viewers)


condition = similar_df['number_of_views'] > 200
similar_df = similar_df[condition]
recommended_movies = similar_df.sort_values('Correlation', ascending=False)
recommended_movies.head(20)

,Correlation,number_of_views
title,,
"Godfather, The (1972)",1.000000,413
"Godfather: Part II, The (1974)",0.683862,209
GoodFellas (1990),0.421477,226
"People vs. Larry Flynt, The (1996)",0.393439,215
Apocalypse Now (1979),0.374378,221
Dead Man Walking (1995),0.360525,299
Psycho (1960),0.336903,239
Field of Dreams (1989),0.309903,212
2001: A Space Odyssey (1968),0.305717,259
